# Training StyleGAN2-ADA on VFSS images
Notebook with the shell commands used to train StyleGan2-ADA on VFSS dataset

In [1]:
# Cloning the StyleGAN3 repository 
# %cd ..
# !git clone https://github.com/NVlabs/stylegan3
# %pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3 scipy

## Optimizing dataset 

In [4]:
%cd /workspace/

/workspace


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
!python stylegan3/dataset_tool.py \
    --source=data-synthesis-vfss/data/images/00000-all-frames-512 \
    --dest=data-synthesis-vfss/data/images/00000-all-frames-512.zip

100%|█████████████████████████████████████| 17496/17496 [04:37<00:00, 63.13it/s]


## Training dataset

Hyperparams set according to the [guide](https://github.com/NVlabs/stylegan3/blob/main/docs/configs.md) provided

In [ ]:
# Train model from scratch
!python stylegan3/train.py \
  --outdir=data-synthesis-vfss/data/training-runs \
  --data=data-synthesis-vfss/data/images/00000-all-frames-512.zip \
  --cfg=stylegan2 \
  --kimg=5000 \
  --snap=25 \
  --gpus=2 \
  --batch=32 \
  --gamma=1.6384 \
  --map-depth=2 \
  --glr=0.0025 \
  --dlr=0.0025 \
  --metrics=none


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan2.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "fused_modconv_default": "inference_only"
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 1.6384,
    "style_mixing_prob": 0.9,
    "pl_weight": 2,
    "pl_no_weight

In [ ]:
# Resume model training

# !python stylegan3/train.py \
#   --resume=data-synthesis-vfss/training-runs/00000-stylegan2-max_constriction-gpus2-batch32-gamma0.8192/network-snapshot-001000.pkl \
#   --outdir=data-synthesis-vfss/training-runs \
#   --data=data-synthesis-vfss/data/images/max_constriction.zip \
#   --cfg=stylegan2 \
#   --gpus=2 \
#   --batch=32 \
#   --gamma=0.8192 \
#   --map-depth=2 \
#   --glr=0.0025 \
#   --dlr=0.0025 \
#   --cbase=16384 \
#   --kimg=4000 \
#   --metrics=none \
#   # --tick=1 \
#   --snap=5

## Generate instances

In [ ]:
# Generate synthetic VFSS images
!python  stylegan3/gen_images.py \
    --outdir=out \
    --trunc=1 \
    --seeds=85,265,297,849 \
    --network=data-synthesis-vfss/training-runs/00000-stylegan2-max_constriction-gpus2-batch32-gamma0.8192/network-snapshot-001000.pkl
    # --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

Loading networks from "data-synthesis-vfss/training-runs/00000-stylegan2-max_constriction-gpus2-batch32-gamma0.8192/network-snapshot-001000.pkl"...
Generating image for seed 85 (0/4) ...
Setting up PyTorch plugin "bias_act_plugin"... /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... /usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Done.
Generating image for seed 265 (1/4) ...
Generating image for seed 297 (2/4) ...
Generating image for seed 849 (3/4) ...


In [ ]:
import os
import matplotlib.pyplot as plt

# display foler's images in 2x2
def plot_images(img_dir, top=2):
    all_img_dirs = os.listdir(img_dir)
    img_files = [os.path.join(img_dir, file) for file in all_img_dirs]

    plt.figure(figsize=(10, 10))

    for idx, img_path in enumerate(img_files):
        plt.subplot(2, 2, idx+1)

        img = plt.imread(img_path)
        plt.tight_layout()
        plt.imshow(img, cmap='gray')
        plt.axis('off')

In [ ]:
# display images in output folder
plot_images('./out')

In [ ]:
plot_images('./out')

In [ ]:
!rm out/*

In [ ]:
# display images in a row
def plot_pics(img_files):
    plt.figure(figsize=(10, 10))

    for i in range(len(img_files)):
        plt.subplot(1, len(img_files), i+1)

        img = plt.imread(img_files[i])
        plt.tight_layout()
        plt.imshow(img, cmap='gray')
        plt.axis('off')

In [ ]:
files = ['out/100-100.png', 'out/1789-1789.png', 'out/100-1789.png']
plot_pics(files)

## Projecting images to latent space

To find the matching latent vector for a given image file, run:

    python projector.py --outdir=out --target=~/mytargetimg.png \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
    
For optimal results, the target image should be cropped and aligned similar to the FFHQ dataset. The above command saves the projection target out/target.png, result out/proj.png, latent vector out/projected_w.npz, and progression video out/proj.mp4. You can render the resulting latent vector by specifying --projected_w for generate.py:
    
    python generate.py --outdir=out --projected_w=out/projected_w.npz \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl    

Reference: [rkuo2000](https://www.kaggle.com/code/rkuo2000/stylegan2-ada)   

### download a picture

In [ ]:
!mkdir -p raw
!wget https://upload.wikimedia.org/wikipedia/commons/6/6d/Shinz%C5%8D_Abe_Official.jpg -O raw/example.jpg

### face alignment

In [ ]:
# face alignment
!python align_images.py raw aligned

In [ ]:
!ls aligned

In [ ]:
files = ['raw/example.jpg', 'aligned/example_01.png']
plot_pics(files)

## Projector

In [ ]:
# projector
!python projector.py --outdir=out --target=aligned/example_01.png \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

In [ ]:
from IPython.display import Video
Video('out/proj.mp4')

In [ ]:
%pip uninstall jax jaxlib -y
%pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# !git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
%pip install ninja

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "./experiments"
NETWORK = "network-snapshot-000120.pkl"
RESUME = os.path.join(EXPERIMENTS, \
                "00003-mainplans1-auto2", NETWORK)
DATA = "../input/mainplans/mainplans1"
SNAP = 35

# Build the command and run it
# cmd = f"python3 ./stylegan2-ada-pytorch/train.py "\
#   f"--snap {SNAP} --resume {RESUME} --mirror=1 --aug=noaug --outdir {EXPERIMENTS} --data {DATA} --gpus=2"

import os
os.system(f"python3 ./stylegan2-ada-pytorch/train.py "\
  f"--snap {SNAP} --resume {RESUME} --mirror=1 --aug=noaug --outdir {EXPERIMENTS} --data {DATA} --gpus=2")

# !{cmd}